<a href="https://colab.research.google.com/github/laibajahangir/-Building-a-basic-Docker-image-and-pushing-it-to-AWS-Elastic-Container-Registry-ECR-/blob/main/AIOps_Log_Analyzer_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # Select raw_logs.txt from your PC

Saving raw_logs.txt to raw_logs.txt


In [ ]:
# Clean logs (remove duplicates/empty lines)
with open('raw_logs.txt', 'r', encoding='utf-8', errors='ignore') as f:
    logs = f.readlines()

cleaned_logs = []
seen = set()
for log in logs:
    log = log.strip()
    if log and log not in seen:
        cleaned_logs.append(log)
        seen.add(log)

with open('cleaned_logs.txt', 'w') as f:
    f.write('\n'.join(cleaned_logs))

In [ ]:
!pip install -q transformers torch

from transformers import AutoTokenizer, AutoModelForCausalLM

# Load TinyLLaMA (fits in 2GB RAM)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

def analyze_log(log):
    prompt = f"""
    Analyze this log line: '{log}'.
    Categorize it as: Error, Security Threat, Performance Issue, or Info.
    Also, identify if it's an anomaly (YES/NO).
    Return format: CATEGORY|ANOMALY
    """
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example output: "Error|YES"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.3 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Keywords for anomalies (customize based on your logs)
anomaly_keywords = ["error", "failed", "attack", "slow", "timeout"]

def is_anomaly(log):
    return any(keyword in log.lower() for keyword in anomaly_keywords)

In [ ]:
from google.colab import files
uploaded = files.upload()  # Select raw_logs.txt from your PC

Saving raw_logs.txt to raw_logs.txt


In [ ]:
# Step 1: Install necessary packages (run once)
!pip install transformers langchain --quiet

# Step 2: Imports
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Step 3: Upload raw logs file
from google.colab import files
uploaded = files.upload()
log_filename = list(uploaded.keys())[0]

with open(log_filename, 'r', encoding='utf-8', errors='ignore') as f:
    raw_logs = f.readlines()

print(f"Total raw logs loaded: {len(raw_logs)}")

# Step 4: Clean logs function
def clean_log(log):
    log = log.strip()
    log = re.sub(r'\s+', ' ', log)
    log = ''.join(c for c in log if c.isprintable())
    return log

cleaned_logs = [clean_log(log) for log in raw_logs if log.strip()]
print(f"Total cleaned logs: {len(cleaned_logs)}")

# Step 5: Load a public Hugging Face model that requires NO authentication
model_name = "gpt2"  # GPT-2 is a public, small model; works without token

print("Loading model (this may take a moment)...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    device=-1  # CPU; set 0 for GPU if available
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 6: Define prompt template for log analysis
prompt_template = """
You are an AI log analyzer. Categorize the following log entry into a category and detect if it contains an anomaly. Respond in this exact format:

<Category>|<True/False>

Log entry: {log}
"""

prompt = PromptTemplate(input_variables=["log"], template=prompt_template)
chain = LLMChain(llm=llm, prompt=prompt)

# Step 7: Analyze logs in batches (simple sequential approach)
def analyze_logs(logs):
    results = []
    for log in logs:
        # Get prompt text
        input_text = prompt.format(log=log)

        # Generate prediction
        outputs = hf_pipeline(input_text, max_new_tokens=30, do_sample=False)
        generated = outputs[0]['generated_text'].strip()

        # Extract response after the prompt text (remove prompt to keep only output)
        response = generated[len(input_text):].strip()

        if '|' in response:
            category, anomaly_str = response.split('|', 1)
            is_anomaly = anomaly_str.strip().lower() == 'true'
        else:
            category = response
            is_anomaly = False

        results.append((log, category.strip(), is_anomaly))
    return results

# Step 8: Run on first 50 logs for demo (adjust as needed)
logs_to_process = cleaned_logs[:50]
print("Analyzing logs...")
results = analyze_logs(logs_to_process)

# Step 9: Show some results
for i, (log, category, anomaly) in enumerate(results[:10]):
    print(f"\nLog #{i+1}:")
    print(f"Log: {log}")
    print(f"Category: {category}")
    print(f"Anomaly: {anomaly}")

# Step 10: Save results to a file
with open("log_analysis_results.txt", 'w', encoding='utf-8') as f:
    for log, cat, anom in results:
        f.write(f"{log}\t{cat}\t{anom}\n")

print("\nDone! Results saved to log_analysis_results.txt")


Saving raw_logs.txt to raw_logs (7).txt
Total raw logs loaded: 9978
Total cleaned logs: 6486
Loading model (this may take a moment)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Analyzing logs...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene


Log #1:
Log: ProviderName: BTHUSB
Category: Log Entry: Name of provider (e-mail address) or service name(s). If you have any questions about your logs please contact us
Anomaly: False

Log #2:
Log: TimeCreated Id LevelDisplayName Message
Category: or any other unique identifier that identifies you as such (e-mail address). If your message is not found within these categories then respond with "
Anomaly: False

Log #3:
Log: ----------- -- ---------------- -------
Category: 
Anomaly: False

Log #4:
Log: 5/21/2025 3:12:14 PM 34 Warning The local adapter does not support an important Low Energy cont...
Category: or any other type of data that is available to you from your computer's network interface (LAN). This may be due either to hardware limitations such
Anomaly: False

Log #5:
Log: 5/21/2025 3:12:14 PM 18 Information Windows cannot store Bluetooth authentication codes (link keys)...
Category: or any other code that is not part of your app's security policy...
Anomaly: False

Log #6:


In [ ]:
import os

project_root = "AIOps_Log_Analyzer_Project"
folders = [
    "data",
    "src",
    "outputs"
]

# Create main and subfolders
os.makedirs(project_root, exist_ok=True)
for folder in folders:
    os.makedirs(os.path.join(project_root, folder), exist_ok=True)

# Files and their content
file_contents = {
    "README.md": """# AIOps Log Analyzer

Ye project real-time machine logs ko analyze karta hai HuggingFace aur LangChain ka use karke.

## Steps to run:

1. Data folder mein apne real logs 'raw_logs.txt' file mein upload karein.
2. Requirements install karein: `pip install -r requirements.txt`
3. 'src/main.py' ko run karein.
4. Output 'outputs/log_analysis_results.txt' mein milega.

Python version: 3.8 ya upar
Dependencies: transformers, langchain, torch, regex
""",

    "requirements.txt": """transformers
langchain
torch
regex
""",

    "src/log_cleaner.py": """import re

def clean_log(log):
    \"\"\"
    Clean a single log entry:
    - Strip leading/trailing spaces
    - Replace multiple spaces/tabs/newlines with single space
    - Remove non-printable characters
    \"\"\"
    log = log.strip()
    log = re.sub(r'\\s+', ' ', log)
    log = ''.join(c for c in log if c.isprintable())
    return log

def clean_logs(logs):
    \"\"\"
    Clean a list of logs by applying clean_log to each.
    \"\"\"
    return [clean_log(log) for log in logs if log.strip()]
""",

    "src/model_setup.py": """from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

def load_model(model_name="gpt2", device=-1):
    print(f"Loading model: {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    hf_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=50,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        device=device  # -1 for CPU, 0 or higher for GPU
    )

    llm = HuggingFacePipeline(pipeline=hf_pipeline)
    print("Model loaded successfully.")
    return llm
""",

    "src/log_analyzer.py": """from langchain import PromptTemplate, LLMChain

def get_prompt_chain(llm):
    template = \"\"\"
You are an AI log analyzer. Categorize the following log entry into a category and detect if it contains an anomaly. Respond in this exact format:

<Category>|<True/False>

Log entry: {log}
\"\"\"
    prompt = PromptTemplate(input_variables=["log"], template=template)
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain, prompt

def analyze_logs(logs, pipeline, prompt):
    results = []
    for log in logs:
        input_text = prompt.format(log=log)
        outputs = pipeline(input_text, max_new_tokens=30, do_sample=False)
        generated = outputs[0]['generated_text'].strip()
        response = generated[len(input_text):].strip()

        if '|' in response:
            category, anomaly_str = response.split('|', 1)
            is_anomaly = anomaly_str.strip().lower() == 'true'
        else:
            category = response
            is_anomaly = False

        results.append((log, category.strip(), is_anomaly))
    return results
""",

    "src/main.py": """import os
from log_cleaner import clean_logs
from model_setup import load_model
from log_analyzer import get_prompt_chain, analyze_logs

def read_logs(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        logs = f.readlines()
    return logs

def save_results(results, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for log, category, is_anomaly in results:
            f.write(f"Log: {log}\\nCategory: {category}\\nAnomaly: {is_anomaly}\\n\\n")

def main():
    data_file = os.path.join("..", "data", "raw_logs.txt")  # relative path from src/
    output_file = os.path.join("..", "outputs", "log_analysis_results.txt")

    print("Reading logs...")
    raw_logs = read_logs(data_file)

    print("Cleaning logs...")
    cleaned_logs = clean_logs(raw_logs)

    print("Loading model...")
    llm = load_model()

    print("Setting up prompt chain...")
    chain, prompt = get_prompt_chain(llm)

    print("Analyzing logs...")
    results = analyze_logs(cleaned_logs, llm.pipeline, prompt)

    print("Saving results...")
    save_results(results, output_file)

    print("Process completed. Results saved in outputs/log_analysis_results.txt")

if __name__ == "__main__":
    main()
"""
}

# Write all files
for filepath, content in file_contents.items():
    full_path = os.path.join(project_root, filepath)
    with open(full_path, "w", encoding="utf-8") as f:
        f.write(content)

print("✅ Project folders and all files created successfully!")


✅ Project folders and all files created successfully!


In [ ]:
!apt-get install tree -y
!tree AIOps_Log_Analyzer_Project


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (107 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
AIOps_Log_Analyzer_Project
├── data
├── outputs
├── README.md
├── requirements.txt
└── src
    ├── log_analyzer.py
    ├── log_cleaner.py
    ├── main.py
    └── model_setup.py

3 directories, 6 files


In [ ]:
from google.colab import files
import shutil
import os

# Step 1: User se local system se file upload karwayen
uploaded = files.upload()

# Step 2: Jo file upload hui hai, uska naam lein
for filename in uploaded.keys():
    print(f'Uploaded file name: {filename}')

    # Step 3: Destination folder path
    dest_folder = "AIOps_Log_Analyzer_Project/data"
    os.makedirs(dest_folder, exist_ok=True)  # Folder agar nahi hai toh bana do

    # Step 4: File ko destination folder me move karo
    shutil.move(filename, os.path.join(dest_folder, "raw_logs.txt"))

print("✅ File uploaded and moved to data/raw_logs.txt successfully.")


Saving raw_logs.txt to raw_logs.txt
Uploaded file name: raw_logs.txt
✅ File uploaded and moved to data/raw_logs.txt successfully.


In [1]:
!git config --global user.name "laibajahangir"
!git config --global user.email "laibakhilji124@gmail.com"


In [8]:
import os

project_root = "/content/AIOps_Log_Analyzer_Project"
folders = [
    "data",
    "src",
    "outputs"
]

# Create main and subfolders
os.makedirs(project_root, exist_ok=True)
for folder in folders:
    os.makedirs(os.path.join(project_root, folder), exist_ok=True)

# Files and their content
file_contents = {
    "README.md": """# AIOps Log Analyzer

Ye project real-time machine logs ko analyze karta hai HuggingFace aur LangChain ka use karke.

## Steps to run:

1. Data folder mein apne real logs 'raw_logs.txt' file mein upload karein.
2. Requirements install karein: `pip install -r requirements.txt`
3. 'src/main.py' ko run karein.
4. Output 'outputs/log_analysis_results.txt' mein milega.

Python version: 3.8 ya upar
Dependencies: transformers, langchain, torch, regex
""",

    "requirements.txt": """transformers
langchain
torch
regex
""",

    "src/log_cleaner.py": """import re

def clean_log(log):
    log = log.strip()
    log = re.sub(r'\\s+', ' ', log)
    log = ''.join(c for c in log if c.isprintable())
    return log

def clean_logs(logs):
    return [clean_log(log) for log in logs if log.strip()]
""",

    "src/model_setup.py": """from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

def load_model(model_name="gpt2", device=-1):
    print(f"Loading model: {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    hf_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=50,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        device=device
    )

    llm = HuggingFacePipeline(pipeline=hf_pipeline)
    print("Model loaded successfully.")
    return llm
""",

    "src/log_analyzer.py": """from langchain import PromptTemplate, LLMChain

def get_prompt_chain(llm):
    template = \"\"\"
You are an AI log analyzer. Categorize the following log entry into a category and detect if it contains an anomaly. Respond in this exact format:

<Category>|<True/False>

Log entry: {log}
\"\"\"
    prompt = PromptTemplate(input_variables=["log"], template=template)
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain, prompt

def analyze_logs(logs, pipeline, prompt):
    results = []
    for log in logs:
        input_text = prompt.format(log=log)
        outputs = pipeline(input_text, max_new_tokens=30, do_sample=False)
        generated = outputs[0]['generated_text'].strip()
        response = generated[len(input_text):].strip()

        if '|' in response:
            category, anomaly_str = response.split('|', 1)
            is_anomaly = anomaly_str.strip().lower() == 'true'
        else:
            category = response
            is_anomaly = False

        results.append((log, category.strip(), is_anomaly))
    return results
""",

    "src/main.py": """import os
from log_cleaner import clean_logs
from model_setup import load_model
from log_analyzer import get_prompt_chain, analyze_logs

def read_logs(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        logs = f.readlines()
    return logs

def save_results(results, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for log, category, is_anomaly in results:
            f.write(f"Log: {log}\\nCategory: {category}\\nAnomaly: {is_anomaly}\\n\\n")

def main():
    data_file = os.path.join("..", "data", "raw_logs.txt")  # relative path from src/
    output_file = os.path.join("..", "outputs", "log_analysis_results.txt")

    print("Reading logs...")
    raw_logs = read_logs(data_file)

    print("Cleaning logs...")
    cleaned_logs = clean_logs(raw_logs)

    print("Loading model...")
    llm = load_model()

    print("Setting up prompt chain...")
    chain, prompt = get_prompt_chain(llm)

    print("Analyzing logs...")
    results = analyze_logs(cleaned_logs, llm.pipeline, prompt)

    print("Saving results...")
    save_results(results, output_file)

    print("Process completed. Results saved in outputs/log_analysis_results.txt")

if __name__ == "__main__":
    main()
"""
}

# Write all files
for filepath, content in file_contents.items():
    full_path = os.path.join(project_root, filepath)
    with open(full_path, "w", encoding="utf-8") as f:
        f.write(content)

print("✅ Project folders and all files created at:", project_root)


✅ Project folders and all files created at: /content/AIOps_Log_Analyzer_Project


In [6]:
# Step 1: Git configuration
!git config --global user.email "laibakhilji124@gmail.com"
!git config --global user.name "Laiba Jahangir"


error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/laibajahangir/aiops-log-analyzer.git'
